# Chemical equation balancer

In [ ]:
import sys
import re
import numpy as np

**a** C + **b** F<sub>2</sub>O<sub>2</sub> &#8594; **c** Fe + **d** CO<sub>2</sub>

C: **a** = **d**

Fe: **2b** = **c**

O: **3b** = **2d**

Annahme a = 1

In [2]:
M = np.array ([[1, 0, 0,-1],
               [0, 2,-1, 0],
               [0, 3, 0,-2],
               [1, 0, 0, 0]], dtype=float)

In [3]:
v = np.array([ 0, 0, 0, 1],dtype=float)

In [4]:
x = np.linalg.solve(M, v)

In [5]:
print('{0:.2f} C + {1:.2f} F\u2082 O\u2082 -> {2:.2f} Fe + {3:.2f} CO\u2082'.format(x[0],x[1],x[2],x[3]))

1.00 C + 0.67 F₂ O₂ -> 1.33 Fe + 1.00 CO₂


**a** H<sub>8</sub>C<sub>6</sub>O<sub>7</sub> + **b** CaCO<sub>3</sub> &#8594; **c** Ca<sub>3</sub> (H<sub>5</sub>C<sub>6</sub>O<sub>7</sub>)<sub>2</sub> + **d** CO<sub>2</sub> + **e** H<sub>2</sub> O

H: **8a** = **10c** + **2e**

C: **6a** + **b** = **12c** + **d**

O: **7a** + **3b** = **14c** + **2d** + **e**

Ca: **b** = **3c**

Annahme: a = 2

Lösung: a=2; b=3; c=1; d=3; e=3

In [6]:
M2 = np.array ([[8, 0,-10, 0,-2],
                [6, 1,-12,-1, 0],
                [7, 3,-14,-2,-1],
                [0, 1, -3, 0, 0],
                [1, 0,  0, 0, 0]], dtype=float)

In [7]:
v2 = np.array([ 0, 0, 0, 0, 2],dtype=float)

In [8]:
x2 = np.linalg.solve(M2, v2)

In [9]:
print('a={0:.2f}; b={1:.2f}; c={2:.2f}; d={3:.2f}; e={4:.2f}'.format(x2[0],x2[1],x2[2],x2[3],x2[4]))

a=2.00; b=3.00; c=1.00; d=3.00; e=3.00


**a** NaOH + **b** Cl<sub>2</sub> &#8594; **c** NaClO + **d** NaCl + **e** H<sub>2</sub>O

Na: **a** = **c** + **d**

O:  **a** = **c** + **e**

H:  **a** = **2e**

Cl: **2b** = **c** + **d**

Annahme: a = 1, erweitert mit 2 -> a=2

Lösung: a=2, b=1, c=1, d=1, e=1


In [10]:
#                a  b  c  d  e
M3 = np.array ([[1, 0,-1,-1, 0],
                [1, 0,-1, 0,-1],
                [1, 0, 0, 0,-2],
                [0, 2,-1,-1, 0],
                [1, 0, 0, 0, 0]], dtype=float)

In [11]:
v3 = np.array([ 0, 0, 0, 0, 2],dtype=float)

In [12]:
x3 = np.linalg.solve(M3, v3)

In [13]:
print('a={0:.2f}; b={1:.2f}; c={2:.2f}; d={3:.2f}; e={4:.2f}'.format(x3[0],x3[1],x3[2],x3[3],x3[4]))

a=2.00; b=1.00; c=1.00; d=1.00; e=1.00


Herstellung von Iod

**a** NaIO<sub>3</sub> + **b** H<sub>2</sub>O + **c** SO<sub>2</sub> &#8594; **d** Na<sub>2</sub>SO<sub>4</sub> + **e** H<sub>2</sub>SO<sub>4</sub> + **f** I<sub>2</sub>

Na: **a** = **2d**

J: **a** = **2f**

O: **3a** + **b** + **2c** = **4d** + **4e**

H: **2b** = **2e**

S: **c** = **d** + **e**

Annahme: a =1

Lösung: a=2; b=4; c=5; d=1; e=4; f=1

In [14]:
#                a  b  c  d  e  f
M4 = np.array ([[1, 0, 0,-2, 0, 0],
                [1, 0, 0, 0, 0,-2],
                [3, 1, 2,-4,-4, 0],
                [0, 2, 0, 0,-2, 0],
                [0, 0, 1,-1,-1, 0],
                [1, 0, 0, 0, 0, 0]], dtype=float)

In [15]:
#               f  e  d  c  b  a
v4 = np.array([ 0, 0, 0, 0, 0, 2],dtype=float)

In [16]:
x4 = np.linalg.solve(M4, v4)

In [17]:
print('a={0:.2f}; b={1:.2f}; c={2:.2f}; d={3:.2f}; e={4:.2f}; f={5:.2f}'.format(x4[0],x4[1],x4[2],x4[3],x4[4],x4[5]))

a=2.00; b=4.00; c=5.00; d=1.00; e=4.00; f=1.00


Verbrennung von Methan

**a** CH<sub>4</sub> + **b** O<sub>2</sub> &#8594; **c** CO<sub>2</sub> + **D** H<sub>2</sub>O<sub>

C: **a** = **c**

H: **4a** = **2d**

O: **2b** = **2c** + **d**

Annahme: a = 1

Lösung: a=1; b=2; c=1; d=2

In [18]:
M5 = np.array ([[1, 0,-1, 0],
                [4, 0, 0,-2],
                [0, 2,-2,-1],
                [1, 0, 0, 0]], dtype=float)

In [19]:
v5 = np.array([0,0,0,1], dtype=float)

In [20]:
x5 = np.linalg.solve(M5, v5)

In [21]:
print('a={0:.2f}; b={1:.2f}; c={2:.2f}; d={3:.2f}'.format(x5[0],x5[1],x5[2],x5[3]))

a=1.00; b=2.00; c=1.00; d=2.00


eq. 12.6 FDS User's Guide (SVN 20596)

In [22]:
M6 = np.array ([[1, 0, 0, 0, 1,-1, 0, 0],
                [4, 0, 0, 2, 0, 0,-2, 0],
                [0, 2, 0, 1, 2,-2,-1, 0],
                [0, 0, 2, 0, 0, 0, 0,-2],
                [1, 0, 0, 0, 0, 0, 0, 0],
                ], dtype=float)

In [23]:
v6 = np.array([0, 0, 0, 0, 1], dtype=float)

In [ ]:
x6 = np.linalg.solve(M6, v6)

Verbrennung von Dieselöl

**a** C<sub>25</sub>H<sub>52</sub> + **b** O<sub>2</sub> &#8594; **c** CO<sub>2</sub> + **d** H<sub>2</sub>O<sub>

C: **25a** = **2c**

H: **52a** = **2d**

O: **2b** = **2c** + **d**

Annahme: a = 1

In [25]:
M7 = np.array ([[25, 0,-1, 0],
                [52, 0, 0,-2],
                [0, 2,-2,-1],
                [1, 0, 0, 0]], dtype=float)

In [26]:
v7 = np.array([0,0,0,1], dtype=float)

In [27]:
x7 = np.linalg.solve(M7, v7)

In [28]:
print('a={0:.2f}; b={1:.2f}; c={2:.2f}; d={3:.2f}'.format(x7[0],x7[1],x7[2],x7[3]))

a=1.00; b=38.00; c=25.00; d=26.00


In [ ]:
#  ==========================
#  = values =
#  ==========================

input_formula = "C1H1.4O0.247N0.000668"
input_Y_CO    = 0.03 # input_Y_CO
input_Y_s     = 0.09 # soot yield
input_X_H     = 0.1   # hydrogen atomic fraction in soot

input_O2      = 0.208
input_N2      = 0.783
input_H2O     = 0.00834
input_CO2     = 0.000387
input_CO      = 0
input_SOOT    = 0

formula = input_formula
y_s  = input_Y_s   # Y_soot
y_CO = input_Y_CO  # Y_CO
X_H  = input_X_H   # hydrogen atomic fraction in soot
bg_1 = 0.0000      # fuel in air
bg_2 = input_O2    # oxygen
bg_3 = input_N2    # nitrogen
bg_4 = input_H2O   # water H2O
bg_5 = input_CO2   # CO2
bg_6 = input_CO    # CO 
bg_7 = input_SOOT  # Soot

#  ==========================
#  = Parse chemical formula =
#  ==========================

C, H, O, N = 0, 0, 0, 0

# Search for C, H, O, N atoms in formula
match = re.search('[cC](\d+\.?\d*)', formula)
if match:
    C = match.group(1)
match = re.search('[hH](\d+\.?\d*)', formula)
if match:
    H = match.group(1)
match = re.search('[oO](\d+\.?\d*)', formula)
if match:
    O = match.group(1)
match = re.search('[nN](\d+\.?\d*)', formula)
if match:
    N = match.group(1)

# If an atom is included with no number following,
# then assign it a value of 1
if (C == 0) and ('C' in formula):
    C = 1
if (H == 0) and ('H' in formula):
    H = 1
if (O == 0) and ('O' in formula):
    O = 1
if (N == 0) and ('N' in formula):
    N = 1

# Convert all atom numbers to floats
C = float(C)
H = float(H)
O = float(O)
N = float(N)

# Throw error if formula does not include C or H
if ((C == 0) or (H == 0)):
    print("C and H must be included in the fuel")
    sys.exit()
#  ==========================================
#  = Chemical equation balance calculations =
#  ==========================================

### Based on work on a previous Matlab script by: ###
# Randall McDermott
# 3-31-11
# fds_simple_chemistry.m
#
# This script is a distilled version of the SIMPLE_CHEMISTRY routine in FDS and may be
# used as a check on reaction coefficients for primitive and lumped species.

# Molecular weights of C, H, O, N
MW_C = 12.0107
MW_H = 1.00794
MW_O = 15.9994
MW_N = 14.0067

# define the element matrix (number of atoms [rows] for each primitive species [columns])

i_fuel            = 0
i_oxygen          = 1
i_nitrogen        = 2
i_water_vapor     = 3
i_carbon_dioxide  = 4
i_carbon_monoxide = 5
i_soot            = 6

E = np.matrix([[C, 0, 0, 0, 1, 1, (1-X_H)],              # C
               [H, 0, 0, 2, 0, 0, X_H    ],              # H
               [O, 2, 0, 1, 2, 1, 0      ],              # O
               [N, 0, 2, 0, 0, 0, 0]     ], dtype=float) # N
     
MW = np.matrix([MW_C, MW_H, MW_O, MW_N]) # primitive species molecular weights

W = E.T * MW.T

# define the volume fractions of the background and fuel

v_0 = np.matrix([bg_1, bg_2, bg_3, bg_4, bg_5, bg_6, bg_7], dtype=float)
v_0 = v_0/np.sum(v_0) # normalize

v_1 = np.matrix([1, 0, 0, 0, 0, 0, 0], dtype=float)
v_1 = v_1/np.sum(v_1) # normalize

# the reaction coefficients for the product primitive species temporarily stored in v_2

v_2 = np.matrix([0, 0, 0, 0, 0, 0, 0], dtype=float)

# compute what we know so far

v_2[0,i_carbon_monoxide] = W.item(i_fuel) / W.item(i_carbon_monoxide) * y_CO
v_2[0,i_soot]            = W.item(i_fuel) / W.item(i_soot) * y_s

# linear system right hand side

b = E * (v_1.T - v_2.T)

# matrix

L = np.column_stack([E*v_0.T, E[:,i_carbon_dioxide], E[:,i_water_vapor], E[:,i_nitrogen]])

# solve the system

x = np.linalg.inv(L)*b

nu_0                    = x.item(0) # background stoichiometric coefficient
v_2.T[i_carbon_dioxide] = x.item(1)
v_2.T[i_water_vapor]    = x.item(2)
v_2.T[i_nitrogen]       = x.item(3)

nu_1 = -1 # fuel stoich coeff
nu_2 = np.sum(v_2) # prod stoich coeff

v_2 = v_2/nu_2 # normalize volume fractions

# display fuel properties

Z2Y = np.vstack([v_0, v_1, v_2])

Z2Y = Z2Y.T

coeff_fuel = Z2Y[0,1] # Fuel

coeff_lhs_1 = Z2Y[0,0] # Fuel
coeff_lhs_2 = Z2Y[1,0] # O2
coeff_lhs_3 = Z2Y[2,0] # N2
coeff_lhs_4 = Z2Y[3,0] # H2O
coeff_lhs_5 = Z2Y[4,0] # CO2
coeff_lhs_6 = Z2Y[5,0] # CO
coeff_lhs_7 = Z2Y[6,0] # C

coeff_rhs_1 = Z2Y[0,2] # Fuel
coeff_rhs_2 = Z2Y[1,2] # O2
coeff_rhs_3 = Z2Y[2,2] # N2
coeff_rhs_4 = Z2Y[3,2] # H2O
coeff_rhs_5 = Z2Y[4,2] # CO2
coeff_rhs_6 = Z2Y[5,2] # CO
coeff_rhs_7 = Z2Y[6,2] # C

if np.min(np.array([coeff_lhs_1, coeff_lhs_2, coeff_lhs_3, coeff_lhs_4, coeff_lhs_5, coeff_lhs_6, coeff_lhs_7,
              coeff_rhs_1, coeff_rhs_2, coeff_rhs_3, coeff_rhs_4, coeff_rhs_5, coeff_rhs_6, coeff_rhs_7])) < 0:
    print("Error: Results inculde negative stoichiometric coefficients")
    sys.exit()

### Balanced chemical equation

In [ ]:
print("&SPEC ID         = 'FUEL_{0}',".format(formula))
print("      FORMULA    = '{0}'/".format(formula))

print("")

i = 1
if coeff_lhs_2 != 0:
    print("      SPEC_ID({1}) = 'OXYGEN',         VOLUME_FRACTION({1})={0:.6f},".format(coeff_lhs_2,i))
    i = i+1
if coeff_lhs_3 != 0:
    print("      SPEC_ID({1}) = 'NITROGEN',       VOLUME_FRACTION({1})={0:.6f},".format(coeff_lhs_3,i))
    i = i+1
if coeff_lhs_4 != 0:
    print("      SPEC_ID({1}) = 'WATER VAPOR',    VOLUME_FRACTION({1})={0:.6f},".format(coeff_lhs_4,i))
    i = i+1
if coeff_lhs_5 != 0:
    print("      SPEC_ID({1}) = 'CARBON DIOXIDE', VOLUME_FRACTION({1})={0:.6f},".format(coeff_lhs_5,i))
    i = i+1
if coeff_lhs_6 != 0:
    print("      SPEC_ID({1}) = 'CARBON MONOXIDE', VOLUME_FRACTION({1})={0:.6f},".format(coeff_lhs_6,i))
    i = i+1
if coeff_lhs_7 != 0:
    print("      SPEC_ID({1}) = 'SOOT',           VOLUME_FRACTION({1})={0:.6f},".format(coeff_lhs_7,i))
    i = i+1
print("      BACKGROUND=.TRUE. /")

print("")

print("&SPEC ID         = 'PRODUCTS',")
i = 1
if coeff_rhs_2 != 0:
    print("      SPEC_ID({1}) = 'OXYGEN',          VOLUME_FRACTION(1})={0:.6f},".format(coeff_rhs_2,i))
    i = i+1
if coeff_rhs_3 != 0:
    print("      SPEC_ID({1}) = 'NITROGEN',        VOLUME_FRACTION({1})={0:.6f},".format(coeff_rhs_3,i))
    i = i+1
if coeff_rhs_4 != 0:
    print("      SPEC_ID({1}) = 'WATER VAPOR',     VOLUME_FRACTION({1})={0:.6f},".format(coeff_rhs_4,i))
    i = i+1
if coeff_rhs_5 != 0:
    print("      SPEC_ID({1}) = 'CARBON DIOXIDE',  VOLUME_FRACTION({1})={0:.6f},".format(coeff_rhs_5,i))
    i = i+1
if coeff_rhs_6 != 0:
    print("      SPEC_ID({1}) = 'CARBON MONOXIDE', VOLUME_FRACTION({1})={0:.6f},".format(coeff_rhs_6,i))
    i = i+1
if coeff_rhs_7 != 0:
    print("      SPEC_ID({1}) = 'SOOT',            VOLUME_FRACTION({1})={0:.6f},".format(coeff_rhs_7,i))
    i = i+1
print("/")
print("")
print("&REAC ID         = 'FUEL_{0}',".format(formula))
print("      FUEL       = 'FUEL_{0}',".format(formula))
print("      SPEC_ID_NU = 'FUEL_{0}','AIR','PRODUCTS',".format(formula))
print("      NU         =-{0:.6f},{1:.6f},{2:.6f},".format(coeff_fuel,nu_0,nu_2))
# ToDo
print("      HEAT_OF_COMBUSTION = 18700 /")